# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 21 2022 2:22PM,253628,17,8931032.1,"Brookfield Pharmaceuticals, LLC",Released
1,Dec 21 2022 2:22PM,253625,17,7370201.2,"Brookfield Pharmaceuticals, LLC",Released
2,Dec 21 2022 2:22PM,253624,17,8931023,"Brookfield Pharmaceuticals, LLC",Released
3,Dec 21 2022 2:21PM,253629,10,0086263496,ISDIN Corporation,Released
4,Dec 21 2022 2:21PM,253629,10,0086263497,ISDIN Corporation,Released
5,Dec 21 2022 2:21PM,253629,10,0086263774,ISDIN Corporation,Released
6,Dec 21 2022 2:17PM,253627,10,Enova-11132,Emerginnova,Released
7,Dec 21 2022 2:17PM,253627,10,DNMD-29351,Emerginnova,Released
8,Dec 21 2022 2:17PM,253627,10,DNMD-29352,Emerginnova,Released
9,Dec 21 2022 2:17PM,253627,10,Eme-106763,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,253625,Released,1
43,253626,Released,1
44,253627,Released,4
45,253628,Released,1
46,253629,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253625,NaN,NaN,1.0
253626,NaN,NaN,1.0
253627,NaN,NaN,4.0
253628,NaN,NaN,1.0
253629,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253547,0.0,5.0,2.0
253560,7.0,8.0,73.0
253563,0.0,1.0,8.0
253564,0.0,0.0,1.0
253570,0.0,0.0,23.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253547,0,5,2
253560,7,8,73
253563,0,1,8
253564,0,0,1
253570,0,0,23


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253547,0,5,2
1,253560,7,8,73
2,253563,0,1,8
3,253564,0,0,1
4,253570,0,0,23


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253547,,5,2
1,253560,7,8,73
2,253563,,1,8
3,253564,,,1
4,253570,,,23


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 21 2022 2:22PM,253628,17,"Brookfield Pharmaceuticals, LLC"
1,Dec 21 2022 2:22PM,253625,17,"Brookfield Pharmaceuticals, LLC"
2,Dec 21 2022 2:22PM,253624,17,"Brookfield Pharmaceuticals, LLC"
3,Dec 21 2022 2:21PM,253629,10,ISDIN Corporation
6,Dec 21 2022 2:17PM,253627,10,Emerginnova
10,Dec 21 2022 2:16PM,253626,10,Emerginnova
11,Dec 21 2022 2:11PM,253622,10,"Senseonics, Incorporated"
12,Dec 21 2022 2:04PM,253620,16,Sartorius Bioprocess Solutions
13,Dec 21 2022 1:46PM,253619,19,"GUSA Granules USA, Inc."
14,Dec 21 2022 1:36PM,253618,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 21 2022 2:22PM,253628,17,"Brookfield Pharmaceuticals, LLC",,,1
1,Dec 21 2022 2:22PM,253625,17,"Brookfield Pharmaceuticals, LLC",,,1
2,Dec 21 2022 2:22PM,253624,17,"Brookfield Pharmaceuticals, LLC",,,1
3,Dec 21 2022 2:21PM,253629,10,ISDIN Corporation,,,3
4,Dec 21 2022 2:17PM,253627,10,Emerginnova,,,4
5,Dec 21 2022 2:16PM,253626,10,Emerginnova,,,1
6,Dec 21 2022 2:11PM,253622,10,"Senseonics, Incorporated",,,1
7,Dec 21 2022 2:04PM,253620,16,Sartorius Bioprocess Solutions,,,1
8,Dec 21 2022 1:46PM,253619,19,"GUSA Granules USA, Inc.",,,1
9,Dec 21 2022 1:36PM,253618,10,ISDIN Corporation,,6,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 2:22PM,253628,17,"Brookfield Pharmaceuticals, LLC",1,,
1,Dec 21 2022 2:22PM,253625,17,"Brookfield Pharmaceuticals, LLC",1,,
2,Dec 21 2022 2:22PM,253624,17,"Brookfield Pharmaceuticals, LLC",1,,
3,Dec 21 2022 2:21PM,253629,10,ISDIN Corporation,3,,
4,Dec 21 2022 2:17PM,253627,10,Emerginnova,4,,
5,Dec 21 2022 2:16PM,253626,10,Emerginnova,1,,
6,Dec 21 2022 2:11PM,253622,10,"Senseonics, Incorporated",1,,
7,Dec 21 2022 2:04PM,253620,16,Sartorius Bioprocess Solutions,1,,
8,Dec 21 2022 1:46PM,253619,19,"GUSA Granules USA, Inc.",1,,
9,Dec 21 2022 1:36PM,253618,10,ISDIN Corporation,4,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 2:22PM,253628,17,"Brookfield Pharmaceuticals, LLC",1,,
1,Dec 21 2022 2:22PM,253625,17,"Brookfield Pharmaceuticals, LLC",1,,
2,Dec 21 2022 2:22PM,253624,17,"Brookfield Pharmaceuticals, LLC",1,,
3,Dec 21 2022 2:21PM,253629,10,ISDIN Corporation,3,,
4,Dec 21 2022 2:17PM,253627,10,Emerginnova,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 2:22PM,253628,17,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
1,Dec 21 2022 2:22PM,253625,17,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
2,Dec 21 2022 2:22PM,253624,17,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
3,Dec 21 2022 2:21PM,253629,10,ISDIN Corporation,3.0,NaN,NaN
4,Dec 21 2022 2:17PM,253627,10,Emerginnova,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 21 2022 2:22PM,253628,17,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
1,Dec 21 2022 2:22PM,253625,17,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
2,Dec 21 2022 2:22PM,253624,17,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
3,Dec 21 2022 2:21PM,253629,10,ISDIN Corporation,3.0,0.0,0.0
4,Dec 21 2022 2:17PM,253627,10,Emerginnova,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6339937,105.0,12.0,0.0
12,507219,0.0,2.0,0.0
16,507198,2.0,0.0,0.0
17,760877,3.0,0.0,0.0
19,1521509,80.0,9.0,7.0
20,507176,2.0,1.0,0.0
21,253603,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6339937,105.0,12.0,0.0
1,12,507219,0.0,2.0,0.0
2,16,507198,2.0,0.0,0.0
3,17,760877,3.0,0.0,0.0
4,19,1521509,80.0,9.0,7.0
5,20,507176,2.0,1.0,0.0
6,21,253603,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,105.0,12.0,0.0
1,12,0.0,2.0,0.0
2,16,2.0,0.0,0.0
3,17,3.0,0.0,0.0
4,19,80.0,9.0,7.0
5,20,2.0,1.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,105.0
1,12,Released,0.0
2,16,Released,2.0
3,17,Released,3.0
4,19,Released,80.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,7.0,0.0,0.0
Executing,12.0,2.0,0.0,0.0,9.0,1.0,0.0
Released,105.0,0.0,2.0,3.0,80.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,7.0,0.0,0.0
1,Executing,12.0,2.0,0.0,0.0,9.0,1.0,0.0
2,Released,105.0,0.0,2.0,3.0,80.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,7.0,0.0,0.0
1,Executing,12.0,2.0,0.0,0.0,9.0,1.0,0.0
2,Released,105.0,0.0,2.0,3.0,80.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()